In [1]:
# check the available memory
print('Available AWS EMER Memory: {}'.format(spark.sparkContext.getConf().get('spark.driver.memory')))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1640238224789_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Available AWS EMER Memory: 11171M

In [2]:
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# import neccessary libraries and packages

import datetime
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window as W

from pyspark.sql.types import (
    StringType,
    IntegerType, 
    DateType, 
    TimestampType,
    )

from pyspark.sql.functions import (
    min as Fmin, max as Fmax, 
    sum as Fsum, round as Fround, 
    
    col, lit, split,
    first, last, 
    desc, asc,
    avg,
    count, countDistinct, approx_count_distinct,
    when, 
    isnull, isnan,
    from_unixtime, date_format, datediff,
    dayofmonth, month, hour, to_date,
    )

# libraries and packages for modeling

from pyspark.ml import Pipeline

from pyspark.ml.feature import (
    StringIndexer, 
    OneHotEncoder, 
    VectorAssembler, 
    StandardScaler
)
from pyspark.ml.feature import (
    OneHotEncoder, 
    OneHotEncoderModel
)

from pyspark.ml.classification import (
    LogisticRegression,
    DecisionTreeClassifier,
    RandomForestClassifier,
    GBTClassifier,
    MultilayerPerceptronClassifier,
    LinearSVC
)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# build a Spark session using the SparkSession APIs

spark = (SparkSession
        .builder
        .appName("Sparkify")
        .getOrCreate())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# create a path variable for the full dataset file
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load and Clean Dataset

In [6]:
def load_data(file_path):
    """
    Loads the raw dataset in Spark.
    
    INPUT:
            (str) - path for datafile
    OUTPUT:
            (PySpark dataframe) - dataframe of raw data
    
    """
    
    print("Loading the dataset ...")
    df = spark.read.json(file_path)
    
    return df


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def clean_data(df):
    """
    Performs basic cleaning operations on the raw data.
        - removes entries with missing userId
        - split the location column in two columns: city and state
        - parses timestamp columns
    
    INPUT:
            (PySpark dataframe) - dataframe of raw data
    OUTPUT:
            (PySpark dataframe) - dataframe of cleaned data
    """
    
    # print message to indicate the start of the process
    print("Cleaning the data ...")
    
    # print a count of rows before cleaning
    initial_records = df.count()
    print("Dataset has {} rows initially.".format(initial_records))
    
    # filter out all the records without userId
    df = df.filter(df.userId != "")
    
    # split the location column in two columns: city and state
    df = df.withColumn("city", split(df["location"], ",").getItem(0)) \
           .withColumn("state", split(df["location"], ",").getItem(1))

    # drop the location column
    df.drop(df.location)
    
    # parse the timestamp columns: registration and ts
    df = df.withColumn("event_ts", from_unixtime(col("ts")/1000.0))
    df = df.withColumn("reg_ts", from_unixtime(col("registration")/1000.0))
    
    # print end of process message
    print("Finished cleaning the data ...")
    
    # print a count of rows after cleaning
    removed_rows = initial_records - df.count()
    print("Cleaned dataset has {} rows, {} rows were removed". \
        format(df.count(), initial_records - df.count()))
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# read the dataset
data_sparkify = load_data(event_data)
# clean the dataset
df = clean_data(data_sparkify)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loading the dataset ...
Cleaning the data ...
Dataset has 26259199 rows initially.
Finished cleaning the data ...
Cleaned dataset has 26259199 rows, 0 rows were removed

## Preprocess Data

In [9]:
def preprocess_data(df):
    
    """
    Prepare the data for modeling via creating several features.
     - firstevent_ts (timestamp) - first time an user is active
     - lastevent_ts (timestamp) - last time an user is active
     - date_ts (date type) - date from timestamp
     - date_reg (timestamp) - registration date
     - init_days_interv (int) - days between registration and first activity
     - tenure_days_interv (int) - days between registration and last activity
     - active_days (int) - days the user has some activity on the platform
     - session_h (float) - session's duration in hours
     
     INPUT:
         df (PySpark dataframe) - cleaned dataframe
     OUTPUT:
         df (PySpark dataframe) - dataframe with the listed features added
    """
    
    # create window: data grouped by userId, time ordered     
    win_user = (W.partitionBy("userId")
            .orderBy("event_ts")
            .rangeBetween(W.unboundedPreceding, W.unboundedFollowing))
    # create window: data grouped by sessionId and userId, time ordered
    win_user_session = (W.partitionBy("sessionId", "userId")
                    .orderBy("event_ts")
                    .rangeBetween(W.unboundedPreceding, W.unboundedFollowing))
    
    # record the first time an user is active
    df = df.withColumn("firstevent_ts", first(col("event_ts")).over(win_user))
    # record the last time an user is active
    df = df.withColumn("lastevent_ts", last(col("event_ts")).over(win_user))

    # extract date from "ts" record
    df = df.withColumn("date_ts",to_date("event_ts")) 
    # extract "registration" date
    df = df.withColumn("date_reg",to_date("reg_ts")) 

    # warmup time = registration time to first event in days
    df = df.withColumn("init_days_interv", datediff(col("firstevent_ts"), col("reg_ts")))
    # tenure time = registration time to last event in days
    df = df.withColumn("tenure_days_interv", datediff(col("lastevent_ts"), col("reg_ts")))
    # active time =  days between the first event and the last event 
    df = df.withColumn("active_days", approx_count_distinct(col("date_ts")).over(win_user))

    # create column that records the individual session's duration in hours
    df = df.withColumn("session_h",
                     (last(df.ts).over(win_user_session) - first(df.ts) \
                      .over(win_user_session))/(1000*3600))  
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# preprocess mini dataset
df_proc = preprocess_data(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# the features in the preprocessed dataframe
df_proc.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- event_ts: string (nullable = true)
 |-- reg_ts: string (nullable = true)
 |-- firstevent_ts: string (nullable = true)
 |-- lastevent_ts: string (nullable = true)
 |-- date_ts: date (nullable = true)
 |-- date_reg: date (nullable

## Feature Engineering

In [12]:
def build_features(df):
    
    """
    Feature engineering to be used in modelling.
    - nr_songs (int) - total number of songs user listened to
    - nr_likes (int) - total number of "Thumbs Up" of the user
    - nr_dislikes (int) - total number of "Thumbs Down" of the user
    - nr_playlist (int) - number of songs added to the playlist
    - nr_friends (int) - number of friends added through "Add Friend"
    - nr_downgrades (int) - total number of visits to "Downgrade" page by the user
    - nr_upgrades (int) - total number of visits to "Upgrade" page by the user
    - nr_home (int) - total number of visits to "Home" page by the user
    - nr_error (int) - total number of errors encountered by the user
    - nr_settings (int) - total number of visits to "Settings" page by the user
    - nr_ads (int) - total number of ads the user got
    - nr_sessions (int) - number of sessions of the user
    - n_acts (int) - total number of actions taken by the user
    - avg_sess_h (float) - average session length in hours
    - acts_per_session (float) - average number of actions per session for the user
    - songs_per_session (float) - average numer of songs listened per session by the user
    - ads_per_session (float) - average number of ads per session, received by user
    - init_days_interv (int) - time interval in days from registration to the first action of the user
    - tenure_days_interv (int) - time interval in days from registration to the last action of the user
    - active_days (int) - number of days the user was active on the platform
    
    - gender (binary) - 1 for F (female), 0 for M (male)
    - level (binary) - 1 for paid, 0 for free
    - downgrade (binary) - 1 for "Submit Downgrade" page visit, 0 otherwise 
    - churn (binary) - 1 for "CAncellation Confirmation" page visit, 0 otherwise
    
    INPUT:
        df (PySpark dataframe) - preprocessed dataframe
        df_feats (PySpark datafrane) - dataframe that contains engineered features
    """
    
    df_feats = df.groupBy("userId") \
        .agg(
    
            # count user's individual actions using all page visits
    
            count(when(col("page") == "NextSong", True)).alias("nr_songs"),
    
            count(when(col("page") == "Thumbs Up", True)).alias("nr_likes"),
            count(when(col("page") == "Thumbs Down", True)).alias("nr_dislikes"),
    
            count(when(col("page") == "Add to Playlist", True)).alias("nr_playlist"),
            count(when(col("page") == "Add Friend", True)).alias("nr_friends"),
    
            count(when(col("page") == "Downgrade", True)).alias("nr_downgrades"),
            count(when(col("page") == "Upgrade", True)).alias("nr_upgrades"),
    
            count(when(col("page") == "Home", True)).alias("nr_home"),
            count(when(col("page") == "Error", True)).alias("nr_error"),
            count(when(col("page") == "Settings", True)).alias("nr_settings"),
    
            count(when(col("page") == "Roll Advert", True)).alias("nr_ads"),
    
            # compute the number of sessions a user is in
            countDistinct("sessionId").alias("nr_sessions"),
    
            # find the total number of actions a user took
            countDistinct("itemInSession").alias("n_acts"),
    
            # compute the average session length in hours
            avg(col("session_h")).alias("avg_sess_h"),
    
            # compute the average number of page actions per sesssion
            (countDistinct("itemInSession")/countDistinct("sessionId")).alias("acts_per_session"),
    
            # compute the average number of songs per session
            (count(when(col("page") == "NextSong",
                        True))/countDistinct("sessionId")).alias("songs_per_session"),
    
            # compute the average number of ads per session
             (count(when(col("page") == "Roll Advert",
                         True))/countDistinct("sessionId")).alias("ads_per_session"),
        
            # days between registration and first activity
            first(col("init_days_interv")).alias("init_days_interv"),
            # the tenure time on the platform: from registration to last event in days
            first(col("tenure_days_interv")).alias("tenure_days_interv"),
            # number of days user visited the platform  
            first(col("active_days")).alias("active_days"),
    
            # encode the gender 1 for F and 0 for M
            first(when(col("gender") == "F", 1).otherwise(0)).alias("gender"),
    
            # encode the level (paid/free) according to the last record
            last(when(col("level") == "paid", 1).otherwise(0)).alias("level"),
    
            # flag those users that downgraded 
            last(when(col("page") == "Downgrade", 1).otherwise(0)).alias("downgrade"),
    
            # create the churn column that records if the user cancelled 
            last(when(col("page") == "Cancellation Confirmation", 1).otherwise(0)).alias("churn"),
            )
    # drop the column "userId" 
    df_feats = df_feats.drop("userId")       
    
    return df_feats

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# build the features dataframe for mini dataset
df_feats = build_features(df_proc)
df_feats.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- nr_songs: long (nullable = false)
 |-- nr_likes: long (nullable = false)
 |-- nr_dislikes: long (nullable = false)
 |-- nr_playlist: long (nullable = false)
 |-- nr_friends: long (nullable = false)
 |-- nr_downgrades: long (nullable = false)
 |-- nr_upgrades: long (nullable = false)
 |-- nr_home: long (nullable = false)
 |-- nr_error: long (nullable = false)
 |-- nr_settings: long (nullable = false)
 |-- nr_ads: long (nullable = false)
 |-- nr_sessions: long (nullable = false)
 |-- n_acts: long (nullable = false)
 |-- avg_sess_h: double (nullable = true)
 |-- acts_per_session: double (nullable = true)
 |-- songs_per_session: double (nullable = true)
 |-- ads_per_session: double (nullable = true)
 |-- init_days_interv: integer (nullable = true)
 |-- tenure_days_interv: integer (nullable = true)
 |-- active_days: long (nullable = true)
 |-- gender: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- downgrade: integer (nullable = true)
 |-- churn: integer (null

### Investigate null values

In [14]:
# drop the null values
df_feats=df_feats.na.drop()
# check for null and missing values
df_feats.select([count(when(isnan(c) | col(c).isNull(), c)) \
           .alias(c) for c in df_feats.columns]) \
           .show(vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-----------------
 nr_songs           | 0   
 nr_likes           | 0   
 nr_dislikes        | 0   
 nr_playlist        | 0   
 nr_friends         | 0   
 nr_downgrades      | 0   
 nr_upgrades        | 0   
 nr_home            | 0   
 nr_error           | 0   
 nr_settings        | 0   
 nr_ads             | 0   
 nr_sessions        | 0   
 n_acts             | 0   
 avg_sess_h         | 0   
 acts_per_session   | 0   
 songs_per_session  | 0   
 ads_per_session    | 0   
 init_days_interv   | 0   
 tenure_days_interv | 0   
 active_days        | 0   
 gender             | 0   
 level              | 0   
 downgrade          | 0   
 churn              | 0

## EDA on Engineered Features

In [15]:
# counts of churn/non_churn users grouped by gender
df_feats.crosstab("churn", "gender").sort("churn_gender").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+----+
|churn_gender|   0|   1|
+------------+----+----+
|           0|8995|8279|
|           1|2656|2347|
+------------+----+----+

In [16]:
# counts of churn/non_churn users grouped by level
df_feats.crosstab("churn", "level").sort("churn_level").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+-----+
|churn_level|   0|    1|
+-----------+----+-----+
|          0|7029|10245|
|          1|1579| 3424|
+-----------+----+-----+

In [17]:
# display statistics for songs played per session
df_feats.describe("songs_per_session").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|songs_per_session|
+-------+-----------------+
|  count|            22277|
|   mean| 65.5269198794409|
| stddev| 41.3827564482615|
|    min|              0.0|
|    max|            579.0|
+-------+-----------------+

In [18]:
# display statistics for actions played per session
df_feats.describe("acts_per_session").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|  acts_per_session|
+-------+------------------+
|  count|             22277|
|   mean|29.963675940246336|
| stddev| 33.17610291712757|
|    min|               0.5|
|    max|             693.0|
+-------+------------------+

In [19]:
# display statistics for average number of ads received per session
df_feats.describe("ads_per_session").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|  ads_per_session|
+-------+-----------------+
|  count|            22277|
|   mean|1.685658016042972|
| stddev|2.047600133034228|
|    min|              0.0|
|    max|             36.0|
+-------+-----------------+

In [20]:
# statistics for the four features
df_feats.describe(["nr_likes", "nr_dislikes", "nr_upgrades", "nr_downgrades"]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+-----------------+------------------+
|summary|         nr_likes|       nr_dislikes|      nr_upgrades|     nr_downgrades|
+-------+-----------------+------------------+-----------------+------------------+
|  count|            22277|             22277|            22277|             22277|
|   mean|51.68851281590879| 10.73807065583337|2.267226287202047|  8.27041343089285|
| stddev|64.58937093224218|12.616020453772165|2.653005101910216|11.600411115347786|
|    min|                0|                 0|                0|                 0|
|    max|              836|               154|               29|               133|
+-------+-----------------+------------------+-----------------+------------------+

### Feature Relevance

In [21]:
def median_diff(df, col, bincol):
    
    """
    Given the values in a column, computes the normalized difference between the medians 
    of the subsets based on values in a categorical column.
    
    INPUT:
        df (Pandas dataframe) - dataframe that contains both columns
        col (Pandas series) - column in df, of integer, float type
        bincol (Pandas series) - column in df, with entries 0, 1
    OUTPUT:
        dictionary: key = nd_col, value = normalized_diff
    """
    median_one = abs(df[df[bincol]==1][col].median())
    median_zero = abs(df[df[bincol]==0][col].median())
    normalized_difference = (median_zero - median_one)/max(median_zero, median_one)
    return {"column": f"nd_{col}",
            "normal_diff": normalized_difference}    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# list of numerical columns to investigate
num_cols = df_feats.toPandas().columns[:20]

# create a Pandas dataframe with the normalized median differences
df_medians = pd.DataFrame(list(map(lambda col: median_diff(df_feats.toPandas(), col, "churn"), num_cols)))
df_medians = df_medians.sort_values(["normal_diff"], ascending = False).reset_index(drop = True)
df_medians.head(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                   column  normal_diff
0             nd_nr_error     1.000000
1   nd_tenure_days_interv     0.291139
2          nd_nr_sessions     0.200000
3             nd_nr_likes     0.193548
4          nd_active_days     0.181818
5          nd_nr_playlist     0.125000
6     nd_init_days_interv     0.121212
7           nd_nr_friends     0.100000
8             nd_nr_songs     0.090586
9              nd_nr_home     0.037037
10              nd_n_acts     0.022936
11          nd_avg_sess_h     0.014503
12         nd_nr_settings     0.000000
13         nd_nr_upgrades     0.000000
14   nd_songs_per_session    -0.031633
15         nd_nr_dislikes    -0.142857
16    nd_acts_per_session    -0.148629
17     nd_ads_per_session    -0.161333
18              nd_nr_ads    -0.166667
19       nd_nr_downgrades    -0.200000

## Modeling

### Split the Data

In [23]:
SPLIT_VALS = [.7, .3]

# split the data into train and test sets

def split_data (df):
    
    """
    Split the dataset into training set and test set.
    """
    train_set, test_set = df.randomSplit(SPLIT_VALS,seed=1234)
    
    return train_set, test_set

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Build Pipeline

In [24]:
# split the features and the label
CAT_FEATURES = ["gender", "level", "downgrade"]
CONT_FEATURES = ["nr_songs", "nr_likes", "nr_dislikes", "nr_playlist", "nr_friends", "nr_downgrades",
                "nr_upgrades", "nr_home", "nr_error", "nr_settings", "nr_ads", "nr_sessions",
                "n_acts", "avg_sess_h", "acts_per_session", "songs_per_session", "ads_per_session",
                "init_days_interv", "tenure_days_interv", "active_days"]
CHURN_LABEL = "churn"


def build_pipeline(classifier):
    """
    Combines all the stages of the processing data. 
    """
    # stages in the pipeline
    stages = [] 
    
    # encode the labels
    label_indexer =  StringIndexer(inputCol=CHURN_LABEL, outputCol="label")
    stages += [label_indexer]
    
    # encode the binary features
    bin_assembler = VectorAssembler(inputCols=CAT_FEATURES, outputCol="bin_features")
    stages += [bin_assembler]
    
    # encode the continuous features
    cont_assembler = VectorAssembler(inputCols = CONT_FEATURES, outputCol="cont_features")
    stages += [cont_assembler]
    # normalize the continuous features
    cont_scaler = StandardScaler(inputCol="cont_features", outputCol="cont_scaler", 
                                 withStd=True , withMean=True)
    stages += [cont_scaler]
    
    # pass all to the vector assembler to create a single sparse vector
    all_assembler = VectorAssembler(inputCols=["bin_features", "cont_scaler"],  
                            outputCol="features")
    stages += [all_assembler]
    
    # add the model to the pipeline
    stages += [classifier]
    
    # create a pipeline
    pipeline = Pipeline(stages=stages)
    
    return pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Build Individual Pipelines

In [25]:
# train a Logistic Regression model
def logistic_regression_pipeline():
    lr_classifier = LogisticRegression(labelCol = "label",
                                       featuresCol = "features")
    return build_pipeline(lr_classifier)

# train a Decision Tree model
def decision_tree_pipeline():
    dt_classifier = DecisionTreeClassifier(labelCol = "label",
                                           featuresCol = "features",
                                           seed=1234)
    return build_pipeline(dt_classifier)

# train a Random Forest model
def random_forest_pipeline():
    rf_classifier = RandomForestClassifier(labelCol = "label",
                                           featuresCol = "features", 
                                           seed=1234)
    return build_pipeline(rf_classifier)

# train a Gradient-boosted Tree model
def gradient_boosted_pipeline():
    gbt_classifier = GBTClassifier(labelCol = "label",
                                   featuresCol = "features",
                                   seed=1234)
    return build_pipeline(gbt_classifier)

# train a Multilayer Perceptron Classifier
def multilayer_perceptron_pipeline():
    # specify layers: 23 (features), two intermediate (8, 4), output 2 (classes)
    layers=[23, 8, 4, 2]
    # create the trainer and set its parameters
    mlpc_classifier = MultilayerPerceptronClassifier(labelCol = "label",
                                                     featuresCol = "features",
                                                     layers=layers,
                                                     seed=1234)
    return build_pipeline(mlpc_classifier)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Build K-Fold Cross Validation and Grid Search Pipelines

In [26]:
# implement K-fold cross validation and grid search 

def grid_search_model(pipeline, param):
    """
    Creates a cross validation object.
    
    INPUT:
        param = grid of parameter
        pipeline = model pipeline 
    
    OUTPUT:
        cv = cross validation object
    """
    evaluator = BinaryClassificationEvaluator()
    cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=param,
                    evaluator=evaluator,
                    numFolds=5,
                    parallelism=2)
    return cv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# model_name = "Logistic_Regression"
def lr_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Logistic Regression
    param_lr = ParamGridBuilder()
    param_lr = param_lr.addGrid(model.regParam, [.01, .1, .5])
    param_lr = param_lr.addGrid(model.elasticNetParam, [.01, .5])
    param_lr = param_lr.build()
    
    print(f"Models trained: {len(param_lr)}")
    
    return grid_search_model(pipeline, param_lr)


# model_name = "Decision Trees"
def dt_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Decision Trees
    param_dt = ParamGridBuilder()
    param_dt = param_dt.addGrid(model.maxDepth, [5, 10, 15, 25])
    param_dt = param_dt.addGrid(model.maxBins, [8, 16, 32])
    param_dt = param_dt.build()
    
    print(f"Models trained: {len(param_dt)}")
    
    return grid_search_model(pipeline, param_dt)


# model_name = "Random Forest"
def rf_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Random Forest
    param_rf = ParamGridBuilder()
    param_rf = param_rf.addGrid(model.maxDepth, [15, 25])
    param_rf = param_rf.addGrid(model.maxBins, [8, 16, 32])
    param_rf = param_rf.addGrid(model.numTrees, [40, 60])
    param_rf = param_rf.build()
    
    print(f"Models trained: {len(param_rf)}")
    
    return grid_search_model(pipeline, param_rf)


# model_name = "Gradient Boosted Trees"
def gbt_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Gradient Boosted Trees
    param_gbt = ParamGridBuilder()
    param_gbt = param_gbt.addGrid(model.maxDepth, [15, 25])
    param_gbt = param_gbt.addGrid(model.maxIter, [10, 20])
    param_gbt = param_gbt.addGrid(model.stepSize, [.05, .1])
    param_gbt = param_gbt.build()
    
    print(f"Models trained: {len(param_gbt)}")
    
    return grid_search_model(pipeline, param_gbt)


# model_name = "Multilayer Perceptron Classifier"
def mlpc_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Multilayer Perceptron
    param_mlpc = ParamGridBuilder()
    param_mlpc = param_mlpc.addGrid(model.stepSize, [.03, .05])
    param_mlpc = param_mlpc.addGrid(model.maxIter, [128, 256])
    param_mlpc = param_mlpc.build()
    
    print(f"Models trained: {len(param_mlpc)}")
    
    return grid_search_model(pipeline, param_mlpc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Build Model Evaluators

In [28]:
# function to compute relevant metrics for binary classification
def conf_metrics(dataset):
    
    """
        Calculates the metrics associated to the confusion matrix.

        INPUT:
            dataset (pyspark.sql.DataFrame) - a dataset that contains
                                labels and predictions
        OUTPUT:
            accuracy (float) - metric
            precision (float) - metric
            recall (float) - metric
            F1 (float) - metric
    """
   

    # calculate the elements of the confusion matrix
    tn = dataset.where((dataset[labelCol]==0) & (dataset[predCol]==0)).count()
    tp = dataset.where((dataset[labelCol]==1) & (dataset[predCol]==1)).count()                   
    fn = dataset.where((dataset[labelCol]==1) & (dataset[predCol]==0)).count()                   
    fp = dataset.where((dataset[labelCol]==0) & (dataset[predCol]==1)).count()
    
    # calculate accuracy, precision, recall, and F1-score
    accuracy = (tn + tp) / (tn + tp + fn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  2 * (precision*recall) / (precision + recall)
    
    return accuracy, precision, recall, f1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
def display_metrics(dataset, roc_cl, pr_cl):
    
    """
    Prints evaluation metrics for the model. 
    
    INPUT:
         dataset (pyspark.sql.DataFrame) - a dataset that contains
                                labels and predictions
    
    """
    
    accuracy = conf_metrics(dataset)[0]
    precision = conf_metrics(dataset)[1]
    recall = conf_metrics(dataset)[2]
    f1 = conf_metrics(dataset)[3]
    
    print("")
    print("Confusion Matrix")
    dataset.groupBy(dataset[labelCol], dataset[predCol]).count().show()
    print("")
    print("accuracy...............%6.3f" % accuracy)
    print("precision..............%6.3f" % precision)
    print("recall.................%6.3f" % recall)
    print("F1.....................%6.3f" % f1)
    print("auc_roc................%6.3f" % roc_cl)
    print("auc_pr.................%6.3f" % pr_cl)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Train, Tune and Evaluate Classifiers

In [30]:
# split the data in training set and test set
train_set, test_set = split_data(df_feats)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Logistic Regression Classifier

In [31]:
print("")
print(f"Training LOGISTIC REGRESSION CLASSIFIER")
print("")

predCol="prediction"
labelCol="label"

# build specific pipeline
pipeline = logistic_regression_pipeline()

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
lr = lr_grid_search(pipeline)

# start timer
start_time = time.time()

# train the model
model_lr = lr.fit(train_set)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_lr = model_lr.bestModel.transform(test_set)

# calculate auc metrics
roc_lr = evaluator.evaluate(predictions_lr, {evaluator.metricName: "areaUnderROC"})
pr_lr = evaluator.evaluate(predictions_lr, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_lr, prec_lr, rec_lr, f1_lr = conf_metrics(predictions_lr)

# print all evaluation metrics
print("")
display_metrics(predictions_lr, roc_lr, pr_lr)
print("")

best_param_lr = list(model_lr.getEstimatorParamMaps()[np.argmax(model_lr.avgMetrics)].values())
print("The best hyperparameter values from the grid:")
print("maxDepth:..........", best_param_lr[0])
print("maxBins:...........", best_param_lr[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-31:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'




Training LOGISTIC REGRESSION CLASSIFIER

Models trained: 6

Training time.........23.539 min


Confusion Matrix
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  597|
|  0.0|       1.0|  129|
|  1.0|       0.0|  952|
|  0.0|       0.0| 4912|
+-----+----------+-----+


accuracy............... 0.836
precision.............. 0.822
recall................. 0.385
F1..................... 0.525
auc_roc................ 0.879
auc_pr................. 0.701

The best hyperparameter values from the grid:
maxDepth:.......... 0.01
maxBins:........... 0.5

### Decision Trees Classifier

In [32]:
print("")
print(f"Training DECISION TREES CLASSIFIER")
print("")

predCol="prediction"
labelCol="label"

# build specific pipeline
pipeline = decision_tree_pipeline()

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
dt = dt_grid_search(pipeline)

# start timer
start_time = time.time()

# train the model
model_dt = dt.fit(train_set)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_dt = model_dt.bestModel.transform(test_set)

# calculate auc metrics
roc_dt = evaluator.evaluate(predictions_dt, {evaluator.metricName: "areaUnderROC"})
pr_dt = evaluator.evaluate(predictions_dt, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_dt, prec_dt, rec_dt, f1_dt = conf_metrics(predictions_dt)

# print all evaluation metrics
print("")
display_metrics(predictions_dt, roc_dt, pr_dt)
print("")

best_param_dt = list(model_dt.getEstimatorParamMaps()[np.argmax(model_dt.avgMetrics)].values())
print("The best hyperparameter values from the grid:")
print("maxDepth:..........", best_param_dt[0])
print("maxBins:...........", best_param_dt[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-32:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'




Training DECISION TREES CLASSIFIER

Models trained: 12

Training time.........23.727 min


Confusion Matrix
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 1392|
|  0.0|       1.0|  196|
|  1.0|       0.0|  157|
|  0.0|       0.0| 4845|
+-----+----------+-----+


accuracy............... 0.946
precision.............. 0.877
recall................. 0.899
F1..................... 0.887
auc_roc................ 0.768
auc_pr................. 0.663

The best hyperparameter values from the grid:
maxDepth:.......... 25
maxBins:........... 32

### Random Forest Classifier

In [33]:
print("")
print(f"Training RANDOM FOREST CLASSIFIER")
print("")

predCol="prediction"
labelCol="label"

# build specific pipeline
pipeline = random_forest_pipeline()

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
rf = rf_grid_search(pipeline)

# start timer
start_time = time.time()

# train the model
model_rf = rf.fit(train_set)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_rf = model_rf.bestModel.transform(test_set)

# calculate auc metrics
roc_rf = evaluator.evaluate(predictions_rf, {evaluator.metricName: "areaUnderROC"})
pr_rf = evaluator.evaluate(predictions_rf, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_rf, prec_rf, rec_rf, f1_rf = conf_metrics(predictions_rf)

# print all evaluation metrics
print("")
display_metrics(predictions_rf, roc_rf, pr_rf)
print("")

best_param_rf = list(model_rf.getEstimatorParamMaps()[np.argmax(model_rf.avgMetrics)].values())
print("The best hyperparameter values from the grid:")
print("maxDepth:..........", best_param_rf[0])
print("maxBins:...........", best_param_rf[1])
print("numTrees:..........", best_param_rf[2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-33:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 11678




Training RANDOM FOREST CLASSIFIER

Models trained: 12

Training time.........162.999 min


Confusion Matrix
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 1068|
|  0.0|       1.0|   50|
|  1.0|       0.0|  481|
|  0.0|       0.0| 4991|
+-----+----------+-----+


accuracy............... 0.919
precision.............. 0.955
recall................. 0.689
F1..................... 0.801
auc_roc................ 0.878
auc_pr................. 0.733

The best hyperparameter values from the grid:
maxDepth:.......... 15
maxBins:........... 32
numTrees:.......... 60

### Gradient Boosted Trees 

In [34]:
print("")
print(f"Training GRADIENT BOOSTED TREES")
print("")

predCol="prediction"
labelCol="label"

# build specific pipeline
pipeline = gradient_boosted_pipeline()

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
gbt = gbt_grid_search(pipeline)

# start timer
start_time = time.time()

# train the model
model_gbt = gbt.fit(train_set)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_gbt = model_gbt.bestModel.transform(test_set)

# calculate auc metrics
roc_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderROC"})
pr_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_gbt, prec_gbt, rec_gbt, f1_gbt = conf_metrics(predictions_gbt)

# print all evaluation metrics
print("")
display_metrics(predictions_gbt, roc_gbt, pr_gbt)
print("")

best_param_gbt = list(model_gbt.getEstimatorParamMaps()[np.argmax(model_gbt.avgMetrics)].values())
print("The best hyperparameter values from the grid:")
print("maxDepth:..........", best_param_gbt[0])
print("maxIter:...........", best_param_gbt[1])
print("stepSize:..........", best_param_gbt[2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-34:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 18679




Training GRADIENT BOOSTED TREES

Models trained: 8

Training time.........395.392 min


Confusion Matrix
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 1394|
|  0.0|       1.0|  153|
|  1.0|       0.0|  155|
|  0.0|       0.0| 4888|
+-----+----------+-----+


accuracy............... 0.953
precision.............. 0.901
recall................. 0.900
F1..................... 0.901
auc_roc................ 0.862
auc_pr................. 0.719

The best hyperparameter values from the grid:
maxDepth:.......... 15
maxIter:........... 20
stepSize:.......... 0.1

### Multilayer Perceptron Classifier

In [ ]:
print("")
print(f"Training MULTILAYER PERCEPTRON CLASSIFIER")
print("")

predCol="prediction"
labelCol="label"

# build specific pipeline
pipeline = multilayer_perceptron_pipeline()

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
mlpc = mlpc_grid_search(pipeline)

# start timer
start_time = time.time()

# train the model
model_mlpc = mlpc.fit(train_set)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_mlpc = model_mlpc.bestModel.transform(test_set)

# calculate auc metrics
roc_mlpc = evaluator.evaluate(predictions_mlpc, {evaluator.metricName: "areaUnderROC"})
pr_mlpc = evaluator.evaluate(predictions_mlpc, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_mlpc, prec_mlpc, rec_mlpc, f1_mlpc = conf_metrics(predictions_mlpc)

# print all evaluation metrics
print("")
display_metrics(predictions_mlpc, roc_mlpc, pr_mlpc)
print("")

best_param_mlpc = list(model_mlpc.getEstimatorParamMaps()[np.argmax(model_mlpc.avgMetrics)].values())
print("The best hyperparameter values from the grid:")
print("stepSize:..........", best_param_mlpc[0])
print("maxIter:......... ..", best_param_mlpc[1])

## Discuss the Best Model

In [ ]:
# create Pandas dataframe with metrics
dict_metrics = {"LinReg": [acc_lr, prec_lr, rec_lr, f1_lr, roc_lr, pr_lr],
                "DecTree": [acc_dt, prec_dt, rec_dt, f1_dt, roc_dt, pr_dt], 
                "RandForest": [acc_rf, prec_rf, rec_rf, f1_rf, roc_rf, pr_rf],
                "GradBoost": [acc_gbt, prec_gbt, rec_gbt, f1_gbt, roc_gbt, pr_gbt],
                "MultiLPerc": [acc_mlpc, prec_mlpc, rec_mlpc, f1_mlpc, roc_mlpc, pr_mlpc],
                "list_metrics" : ["accuracy", "precision", "recall", "f1_score", "auc_roc", "auc_pr"]
               }
df_mets = pd.DataFrame.from_dict(dict_metrics).set_index("list_metrics")
df_mets.round(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              LinReg  DecTree  RandForest  GradBoost  MultiLPerc
list_metrics                                                    
accuracy       0.836    0.946       0.919      0.953       0.906
precision      0.822    0.877       0.955      0.901       0.855
recall         0.385    0.899       0.689      0.900       0.720
f1_score       0.525    0.887       0.801      0.901       0.782
auc_roc        0.879    0.768       0.878      0.862       0.923
auc_pr         0.701    0.663       0.733      0.719       0.839